In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as sla
import matplotlib.pyplot as plt

In [2]:
def logit(x, w):
    return np.dot(x, w)

def sigmoid(h):
    return 1. / (1 + np.exp(-h))

In [10]:
def generate_batches(X, y, batch_size):
    assert len(X) == len(y)
    np.random.seed(42)
    X = np.array(X)
    y = np.array(y)
    perm = np.random.permutation(len(X))

    n_batches = len(X) // batch_size
    
    for batch_idx in range(n_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        X_batch = X[perm[start_idx:end_idx]]
        y_batch = y[perm[start_idx:end_idx]]
        yield X_batch, y_batch
    

In [11]:
class MyLogisticRegression(object):
    def __init__(self):
        self.w = None

    def fit(self, X, y, epochs=10, lr=0.1, batch_size=100):
        l, n = X.shape
        if self.w is None:
            np.random.seed(42)
            self.w = np.random.randn(n + 1)

        X_train = np.concatenate((np.ones((l, 1)), X), axis=1)

        losses = []


        for i in range(epochs):
            for X_batch, y_batch in generate_batches(X_train, y, batch_size):
                

                predictions = self._predict_proba_internal(X_batch)
                loss = self.__loss(y_batch, predictions)

                assert (np.array(loss).shape == tuple()), "!"

                losses.append(loss)
                grad = self.get_grad(X_batch, y_batch, predictions)
                self.w = self.w - lr * grad

        return losses

    def get_grad(self, X_batch, y_batch, predictions):
        grad_basic = np.dot(X_batch.T, predictions - y_batch)

        return grad_basic

    def predict_proba(self, X):
        l, n = X.shape
        X_ = np.concatenate((np.ones((l, 1)), X), axis=1)
        return sigmoid(X_ @ self.w)

    def _predict_proba_internal(self, X):

        return sigmoid(X @ self.w)

    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

    def get_weights(self):
        return self.w.copy()

    def __loss(self, y, p):
        p = np.clip(p, 1e-10, 1 - 1e-10)
        return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))


In [12]:
class MyElasticLogisticRegression(MyLogisticRegression):
    def __init__(self, l1_coef, l2_coef):
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.w = None

    def get_grad(self, X_batch, y_batch, predictions):

        grad_basic = np.dot(X_batch.T, predictions - y_batch)

        grad_l1 = self.l1_coef * np.sign(self.w[1:])
        grad_l1 = np.concatenate(([0], grad_l1))

        grad_l2 = 2 * self.l2_coef * self.w[1:]
        grad_l2 = np.concatenate(([0], grad_l2))

        return grad_basic + grad_l1 + grad_l2

In [16]:
data = pd.read_csv('/home/daniil/Загрузки/MNIST.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X = X[(y == 0) | (y == 1)]
y = y[(y == 0) | (y == 1)]

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    StandardScaler(),
    MyElasticLogisticRegression(l1_coef=0.1, l2_coef=0.1)
)
scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
mean_accuracy = np.mean(scores)
print(f"Mean accuracy of Logistic Regression for two classes is {mean_accuracy}")

/tmp/ipykernel_3189/186220842.py:5: RuntimeWarning: overflow encountered in exp
  return 1. / (1 + np.exp(-h))


Mean accuracy of Logistic Regression for two classes is 0.994781943785266
